In [1]:
import zmq
import numpy as np
import threading

import zmq
import numpy as np

def create_push_socket(ip, port):
    context = zmq.Context()
    socket = context.socket(zmq.PUSH)
    socket.connect(f"tcp://{ip}:{port}")
    
    return socket

def create_pull_socket(ip, port):
    context = zmq.Context()
    socket = context.socket(zmq.PULL)
    socket.bind(f"tcp://{ip}:{port}")
    
    return socket

def send_number_matrix(push_socket, number, matrix):

    msg = {"number": number, "matrix": matrix.tolist()}

    push_socket.send_json(msg)
    
def receive_number(pull_socket):
    msg = pull_socket.recv_json()
    
    return float(msg["number"])
    
    
def aggregate_numbers_matrices(pull_socket, n):
    local_matrices = []
    local_numbers = []

    for _ in range(n):
        msg = pull_socket.recv_json()
        matrix = np.array(msg["matrix"])
        number = float(msg["number"])
        local_matrices.append(matrix)
        local_numbers.append(number)

    mean_matrix = np.mean(local_matrices, axis=0)
    mean_number = np.mean(local_numbers)

    return mean_number, mean_matrix

def start_aggregation_loop(pull_socket, n, callback):
    def loop():
        while True:
            result = aggregate_numbers_matrices(pull_socket, n)
            callback(result)

    t = threading.Thread(target=loop, daemon=True)
    t.start()
    
    
def on_aggregated(result):
    # TODO: Send back!
    mean_number, mean_matrix = result
    print("Aggregation complete")
    print("Mean number:", mean_number)
    print("Mean matrix:\n", mean_matrix)

In [2]:
push = create_push_socket('localhost', 9205)

In [8]:
send_number_matrix(push, 4., np.random.rand(1,2))

In [2]:
import json

In [3]:
with open('hyperparameters/test_1.json') as f:
    x = json.load(f)

In [4]:
import pprint

In [5]:
pprint.pprint(x)

{'clients': {'lr': 0.001,
             'network': [{'data_path': '/home/mtheologitis/FDA-Opt-Sys/data/glue/mrpc/c2_0.pth',
                          'id': 0,
                          'ip': 'localhost',
                          'port': 9200},
                         {'data_path': '/home/mtheologitis/FDA-Opt-Sys/data/glue/mrpc/c2_1.pth',
                          'id': 1,
                          'ip': 'localhost',
                          'port': 9201}]},
 'dataset': {'batch_size': 8,
             'dirichlet_alpha': 1,
             'name': 'mrpc',
             'path': 'glue'},
 'model': {'checkpoint': 'prajjwal1/bert-tiny', 'num_labels': 2},
 'server': {'network': {'ip': 'localhost',
                        'ip_pull_socket': 'localhost',
                        'port': 8083,
                        'port_pull_socket': 9101},
            'strategy': {'eta': 0.0001, 'fda': True, 'name': 'FedAdam'}},
 'training': {'clients_per_round': 2,
              'local_epochs': 1,
              

In [7]:
x['clients']['network'][0]['port'] = 9199

In [ ]:
9199

In [8]:
with open('hyperparameters/test_1.json', 'w', encoding='utf-8') as f:
    json.dump(x, f, ensure_ascii=False)